# Neonatal Mortality V/s Nursing personnel
   Here I am trying to find correlation between neonatal(Relating to newborn baby) mortality rate and nursing and midwifery personnel. Dataset is of countries hence to do regional analysis I mapped it with their region. Neonatal mortality rate is of 2019 and nursing personnel data is from 2018.

### Importing libraries and importing datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

nmr = pd.read_csv("../input/who-worldhealth-statistics-2020-complete/neonatalMortalityRate.csv").drop(columns=['Dim1'])
continents = pd.read_csv("../input/country-mapping-iso-continent-region/continents2.csv")
nmp = pd.read_csv("../input/who-worldhealth-statistics-2020-complete/nursingAndMidwife.csv")

### Making a function that maps countries with their region

In [ ]:
def add_region(year, df):
    regional_df = df[df['Period'] == year]
    regional_df = regional_df.drop(columns=['Period', 'Indicator'])
    regional_df.replace(r'(\s.*\]$)', '', regex=True, inplace=True)
    regional_df['First Tooltip'] = regional_df['First Tooltip'].astype(float)

    regional_df = regional_df.merge(continents[['name', 'region']], 
        left_on='Location', right_on='name').drop(columns=['name'])

    regional_df.set_index(['region'], inplace=True)
    regional_df.loc['Asia', 'mean'] = regional_df.loc['Asia', 'First Tooltip'].mean()
    regional_df.loc['Europe', 'mean'] = regional_df.loc['Europe', 'First Tooltip'].mean()
    regional_df.loc['Africa', 'mean'] = regional_df.loc['Africa', 'First Tooltip'].mean()
    regional_df.loc['Americas', 'mean'] = regional_df.loc['Americas', 'First Tooltip'].mean()
    regional_df.loc['Oceania', 'mean'] = regional_df.loc['Oceania', 'First Tooltip'].mean()
    # print(regional_df.sort_values(['region']).tail())
    data = regional_df.reset_index()

    return data

### Merging dataframes

In [ ]:
neonatal = add_region(2019, nmr).sort_values(['mean'], ascending=False)
nurse = add_region(2018, nmp).sort_values(['mean'])
nurse['nmean'] = nurse['mean']
merged_df = neonatal.merge(nurse[['region', 'nmean']], left_on='region', right_on='region')

### Plotting the data

In [ ]:
def plot_graph():

    fig, ax = plt.subplots(4,1, figsize=(6.4,7))
    ax1 = plt.subplot2grid((4,1), (0,0), rowspan=1, colspan=1)
    ax2 = plt.subplot2grid((4,1), (1,0), rowspan=3, colspan=1, sharex=ax1)
    # ax1.set_title('Neonatal mortality rate in 2019\n(per 1000 live births)',fontsize=18)
    ax1.bar(merged_df['region'], merged_df['mean'], width=0.7, color='#fc4f30')
    ax2.bar(merged_df['region'], merged_df['nmean'], width=0.7, color='#008fd5')
    ax2.invert_yaxis()
    for p in ax1.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy() 
        ax1.annotate(round(height, 2), (x + width/2, y+height+0.3), ha='center')
    for p in ax2.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy() 
        ax2.annotate(round(height, 2), (x + width/2, y+height-1.5), ha='center')
    plt.setp(ax1.get_yticklabels(), visible=False)
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax2.get_yticklabels(), visible=False)
    ax1.grid(False)
    ax2.grid(False)
    ax1.text(1,24,'Neonatal mortality rate in 2019\n                 (per 1000 live births)',
     fontsize=14, color='#fc4f30', fontweight='bold')
    ax2.text(-0.5,90,'Nursing and midwifery personnel\nin 2018 (per 10,000 people)',
     fontsize=14, color='#008fd5', fontweight='bold')
    ax2.text(-0.5,95, 'Prepared by: twitter.com\meet_hp\nSource: kaggle.com/utkarshxy/who-world-health-statistics-2020-complete', fontsize=6)
    ax2.xaxis.tick_top()
    plt.show()
    
plot_graph()